# PARSING SECTION

In [ ]:
import pandas as pd

# Path to your Excel file
file_path = "all-schemes-monthly-portfolio---as-on-28th-february-2023.xlsx"

# Load the workbook
xls = pd.ExcelFile(file_path)

In [ ]:
!curl -O https://www.sbimf.com/docs/default-source/scheme-portfolios/all-schemes-monthly-portfolio---as-on-28th-february-2023.xlsx

In [ ]:
sheet_name = 'SLMF'

In [ ]:
# snippet fining the fund name
temp = pd.read_excel(file_path, sheet_name=sheet_name, nrows=5)
temp = temp.iloc[:,[2,3]]
fund_name = temp.iloc[1,1]
fund_name


In [ ]:
# read the data from dataframes
df = xls.parse(sheet_name, header=5)
df.dropna(how='all', inplace=True) #drop completly blank rows , make changes directly to the df -- inplace = true
nos_of_cols =  df.shape[1]


In [ ]:
nos_of_cols

In [ ]:
# selecting cols based on number of cols 
# still to test if the logic can be flaky
if nos_of_cols == 11:
    selected_df = df.iloc[:, [2, 3, 4, 5, 6, 7,8,9,10]] # specify the cols required
if nos_of_cols == 12:
    selected_df = df.iloc[:, [2, 3, 4, 5, 6, 7,8,9,10,11]] # specify the cols required

In [ ]:
selected_df.info()

In [63]:
sections = {

    "EQUITY & EQUITY RELATED": {
        "rename": "equities",
        "sublist":[
            "a) Listed/awaiting listing on Stock Exchanges",
            "b) Unlisted",
            "c) Foreign Securities and /or overseas ETF"
            ]    
    },
    "DEBT INSTRUMENTS": {
        "sublist":[
            "a) Listed/awaiting listing on the stock exchanges",
            "b) Privately Placed/Unlisted",
            "c) Securitised Debt Instruments",
            "d) Central Government Securities",
            "e) State Government Securities"
        ]
    },
    "MONEY MARKET INSTRUMENTS": {
        "sublist":[
            "a) Commercial Paper",
            "b) Certificate of Deposits",
            "c) Treasury Bills",
            "d) Bills Re- Discounting",
            "e) STRIPS"
        ]
    },
    "OTHERS": {
            "sublist":[
                "a) Mutual Fund Units / Exchange Traded Funds",
                "b) Gold",
                "c) Short Term Deposits",
                "d) Term Deposits Placed as Margins",
                "e) TREPS / Reverse Repo Investments"
            ]
    },
    "Other Current Assets / (Liabilities)":{
            "sublist":[]
    },
    "DERIVATIVES":{
        "sublist":[
            "Stock Futures"
        ]
    }
    
    
}

In [ ]:
selected_df.info()

In [79]:
# rename headers 
temp = selected_df.copy()

#temp = temp.columns.str.replace(r'\s+', ' ', regex=True).str.strip()


temp.columns = temp.columns.str.replace(r'YTC.*', 'ytc_percent', regex=True)
temp.columns = temp.columns.str.replace(r'YTM.*', 'ytm_percent', regex=True)
temp.columns = temp.columns.str.replace(r'Notes.*', 'notes_symbols', regex=True)
temp.columns = temp.columns.str.replace(r'Rating.*', 'sector', regex=True)
temp.columns = temp.columns.str.replace(r'Market[\s\S]*', 'market_value_lakhs', regex=True)
temp.columns = temp.columns.str.replace(r'Name of.*', 'inst_name', regex=True)

temp = temp.rename(columns={
     "ISIN":"isin",
     "Quantity":"quantity",
     "% to AUM":"aum_percent",
     "ESG Score":"esg_score"
})

# adding missing cols 


temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110 entries, 1 to 134
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_name           109 non-null    object 
 1   isin                68 non-null     object 
 2   sector              70 non-null     object 
 3   quantity            68 non-null     object 
 4   market_value_lakhs  92 non-null     object 
 5   aum_percent         92 non-null     object 
 6   ytm_percent         1 non-null      object 
 7   ytc_percent         0 non-null      float64
 8   notes_symbols       4 non-null      object 
dtypes: float64(1), object(8)
memory usage: 8.6+ KB


In [75]:
# import re
# # cleanse row
# temp['quantity'] = temp['quantity'].apply(
#     lambda x: re.search(r'\((\d+)\)', str(x)).group(1) if re.search(r'\((\d+)\)', str(x)) else x
# )

In [80]:
# core logic to create a KV data structure
ommit_rows = ['','Notes & Symbols :-','GRAND TOTAL (AUM)','Name of the Instrument','Derivatives Total']
section_dataframes = {}
current_section = None
section_start = None

current_section_type = None
current_sub_section_type = None

for idx, row in temp.iterrows():
    holdings = None
    
    if row[0] == 'Notes & Symbols :-':
        break;
    else:
        if row[0] in sections:
            if current_section_type != row[0]:
                current_section_type = row[0]
                current_sub_section_type = ''
        elif row[0] in sections[current_section_type]['sublist']:
            current_sub_section_type = row[0]
        elif row[0] not in ommit_rows :
            holdings = {
                 'inst_name':row['inst_name'] ,  
                 'isin':row['isin'],
                 'sector':row['sector'],
                 'quantity':row['quantity'],
                 #'quantity':row['quantity'].str.extract(r'\((\d+)\)')
                 'market_value_lakhs':row['market_value_lakhs'],
                 'aum_percent':row['aum_percent'],
                 # 'esg_score':row['esg_score'] if ,
                 'esg_score':row.get('esg_score','') or '',
                 'notes_symbols':row['notes_symbols'],
                 'ytm_percent':row['ytm_percent'],
                 'ytc_percent':row['ytc_percent']
            }
            section_dataframes.setdefault(current_section_type,{}).setdefault(current_sub_section_type,[]).append(holdings)
        
    

/var/folders/5t/hzhlndks1l3_sl0r3gcllxx00000gn/T/ipykernel_91568/2474830119.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] == 'Notes & Symbols :-':
/var/folders/5t/hzhlndks1l3_sl0r3gcllxx00000gn/T/ipykernel_91568/2474830119.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] in sections:
/var/folders/5t/hzhlndks1l3_sl0r3gcllxx00000gn/T/ipykernel_91568/2474830119.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc

In [ ]:
section_dataframes

In [81]:
# conversion of flat_rows to flat_df
flat_rows = []

for section_type, subsections in section_dataframes.items():
    for sub_section_type, holdings_list in subsections.items():
        for holding in holdings_list:
            flat_rows.append({
                'section_type': section_type,
                'sub_section_type': sub_section_type,
                **holding  # expands the holding dict into flat columns
            })


final_df = pd.DataFrame(flat_rows)

In [82]:
final_df

,section_type,sub_section_type,inst_name,isin,sector,quantity,market_value_lakhs,aum_percent,esg_score,notes_symbols,ytm_percent,ytc_percent
0,EQUITY & EQUITY RELATED,a) Listed/awaiting listing on Stock Exchanges,ICICI Bank Ltd.,INE090A01021,Banks,6400000.0,54710.400,5.79,,NaN,NaN,NaN
1,EQUITY & EQUITY RELATED,a) Listed/awaiting listing on Stock Exchanges,HDFC Bank Ltd.,INE040A01034,Banks,2840000.0,45428.640,4.81,,NaN,NaN,NaN
2,EQUITY & EQUITY RELATED,a) Listed/awaiting listing on Stock Exchanges,Infosys Ltd.,INE009A01021,IT - Software,2540000.0,37783.770,4,,NaN,NaN,NaN
3,EQUITY & EQUITY RELATED,a) Listed/awaiting listing on Stock Exchanges,ITC Ltd.,INE154A01025,Diversified FMCG,8300000.0,31266.100,3.31,,NaN,NaN,NaN
4,EQUITY & EQUITY RELATED,a) Listed/awaiting listing on Stock Exchanges,State Bank of India,INE062A01020,Banks,5600000.0,29276.800,3.1,,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
69,OTHERS,e) TREPS / Reverse Repo Investments,Total,NaN,NaN,NaN,59206.530,6.27,,NaN,NaN,NaN
70,Other Current Assets / (Liabilities),,Margin amount for Derivative positions,NaN,NaN,NaN,10160.000,1.08,,NaN,NaN,NaN
71,Other Current Assets / (Liabilities),,Net Receivable / Payable,NaN,NaN,NaN,-381.250,-0.07,,NaN,NaN,NaN
72,Other Current Assets / (Liabilities),,Total,NaN,NaN,NaN,9778.750,1.01,,NaN,NaN,NaN


In [ ]:


# renaming sections in the final df

final_df['section_type'] = final_df['section_type'].replace(
    {
        'EQUITY & EQUITY RELATED': 'equities',
        'OTHERS': 'other',
        'Other Current Assets / (Liabilities)': 'assets_or_liabilities'
    }
)

## subsections
final_df['sub_section_type'] = final_df['sub_section_type'].replace(
    {
        'a) Listed/awaiting listing on Stock Exchanges': 'listed_on_exchange',
        'b) Unlisted': 'unlisted',
        'c) Foreign Securities and /or overseas ETF': 'overseas_investments',
        'e) TREPS / Reverse Repo Investments': 'treps_reserve_repo'
    }
)

#marking as asset or liability
final_df['type'] = np.where(
    (final_df['market_value_lakhs'] < 0 ),
    'liability',  # value if condition is True
    'asset'          # value if condition is False
)